In [1]:
import numpy as np
from gurobipy import*
import math
m=Model('TRE example')
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-12


In [2]:
## S storage locations
## K picking stations
## R robots
## j tasks
ns = 30
nk = 2
nr = 2
nj = 4
npod = 8
nn = npod+nj
stoloc_x = np.zeros(ns)
stoloc_y = np.zeros(ns)
for i in range(ns):
    stoloc_x[i] = (i%5)*2 + 1
    stoloc_y[i] = math.floor(i/5)*2 + 1
staloc_x = np.array([4, 6])
staloc_y = np.array([10, 10])
charge_x = 5
charge_y = 8
stoloc_x = np.append(stoloc_x, charge_x)
stoloc_y = np.append(stoloc_y, charge_y)
J1 = [1, 2]
J2 = [3, 4]
listJ = [[1, 2],[3, 4]]
J = [1, 2, 3, 4]
JE = [1, 2, 3, 4]
JF = [1, 2, 3, 4]
jo = [0]
jd = [5]
JI = [6, 7, 8, 9, 10, 11, 12, 13]
pi = [-1, 0, 0, 1, 1, -1]
D = [0, 7, 10, 5, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b = [-1, 6, 7, 8, 9, 0]
n = [nj+1, -1, -1, -1, -1, -1]
rou = np.zeros(nj+npod+2)
rou[0] = ns
rou[5] = ns
for i in range(npod):
    rou[i+nj+2] = i

# S最后一个是charge location
Dsetup = 0
dsj = np.zeros([ns+1, nj+2])
for i1 in range(ns+1):
    for i2 in range(1, nj+1):
        dsj[i1,i2] = np.absolute(staloc_x[pi[i2]] - stoloc_x[i1]) + np.absolute(staloc_y[pi[i2]] - stoloc_y[i1])

djs = np.zeros([nj+2, ns+1])
for i1 in range(1, nj+1):
    for i2 in range(ns+1):
        djs[i1,i2] = dsj[i2, i1]

dss = np.zeros([ns+1, ns+1])
for i1 in range(ns+1):
    for i2 in range(ns+1):
        dss[i1,i2] = np.absolute(stoloc_x[i1] - stoloc_x[i2]) + np.absolute(stoloc_y[i1] - stoloc_y[i2])

In [3]:
# ## S storage locations
# ## K picking stations
# ## R robots
# ## j tasks
# ns = 30
# nk = 2
# nr = 4
# nj = 10
# npod = 24
# nn = npod+nj
# stoloc_x = np.zeros(ns)
# stoloc_y = np.zeros(ns)
# for i in range(ns):
#     stoloc_x[i] = (i%5)*2 + 1
#     stoloc_y[i] = math.floor(i/5)*2 + 1
# staloc_x = np.array([4, 6])
# staloc_y = np.array([10, 10])
# charge_x = 5
# charge_y = 8
# stoloc_x = np.append(stoloc_x, charge_x)
# stoloc_y = np.append(stoloc_y, charge_y)

In [4]:
# J1 = list(range(1, int(nj/nk)+1))
# J2 = list(range(int(nj/nk)+1, 2*int(nj/nk)+1))
# # J3 = list(range(2*int(nj/nk)+1, 3*int(nj/nk)+1))
# # listJ = [J1, J2, J3]
# listJ = [J1, J2]
# # J = J1+J2+J3
# J = J1+J2
# JE = JF = J
# jo = [0]
# jd = [nj+1]
# JI = list(range(nj+1,nj+npod+1))
# pi = [-1]
# for i in range(int(nj/nk)):
#     pi.append(0)
# for i in range(int(nj/nk)):
#     pi.append(1)
# # for i in range(int(nj/nk)):
# #     pi.append(2)
# pi.append(-1)
# D = [0]
# for i in range(nj):
#     D.append(np.random.randint(5,10))
# for i in range(npod+1):
#     D.append(0)
# b = [-1]
# for i in range(1, nj+1):
#     b.append(i+nj+1)
# b.append(0)
# n = [nj+1]
# for i in range(1, nj+2):
#     n.append(-1)

In [5]:
# rou = np.zeros(nj+npod+2)
# rou[int(jo[0])] = ns
# rou[int(jd[0])] = ns
# for i in range(npod):
#     rou[i+nj+2] = i

# # S最后一个是charge location
# Dsetup = 0
# dsj = np.zeros([ns+1, nj+2])
# for i1 in range(ns+1):
#     for i2 in range(1, nj+1):
#         dsj[i1,i2] = np.absolute(staloc_x[pi[i2]] - stoloc_x[i1]) + np.absolute(staloc_y[pi[i2]] - stoloc_y[i1])

# djs = np.zeros([nj+2, ns+1])
# for i1 in range(1, nj+1):
#     for i2 in range(ns+1):
#         djs[i1,i2] = dsj[i2, i1]

# dss = np.zeros([ns+1, ns+1])
# for i1 in range(ns+1):
#     for i2 in range(ns+1):
#         dss[i1,i2] = np.absolute(stoloc_x[i1] - stoloc_x[i2]) + np.absolute(stoloc_y[i1] - stoloc_y[i2])

In [6]:
M = 1e3

In [7]:
S = range(ns+1)
R = range(nr)
K = range(nk)
Jio = J+JI+jo
Jid = J+JI+jd
Jod = J+jo+jd
Jiod = J+JI+jo+jd
Jd = J+jd
Jo = J+jo
iod = JI+jo+jd
od = jo+jd

In [8]:
u = {}
for j in J+JI:
    for r in R:
        u[j,r] = m.addVar(vtype=GRB.INTEGER,name="u_%d_%d"%(j,r))

In [9]:
x = {}
for s in S:
    for j in Jiod:
        x[s,j] = m.addVar(vtype=GRB.BINARY,name="x_%d_%d"%(s,j))
w = {}
for r in R:
    for j in Jiod:
        w[r,j] = m.addVar(vtype=GRB.BINARY,name="w_%d_%d"%(r,j))
v = {}
for s1 in S:
    for s2 in S:
        for j1 in Jio:
            for j2 in Jid:
                v[s1,s2,j1,j2] = m.addVar(vtype=GRB.BINARY,name="v_%d_%d_%d_%d"%(s1,s2,j1,j2))

In [10]:
Cmax = m.addVar(vtype=GRB.CONTINUOUS,name="Cmax")

z = {}
for r in R:
    for j1 in Jio:
        for j2 in Jid:
            z[r,j1,j2] = m.addVar(vtype=GRB.BINARY,name="z_%d_%d_%d"%(r,j1,j2))
g = {}
for s in S:
    for j in Jiod:
        g[s,j] = m.addVar(vtype=GRB.CONTINUOUS,name="g_%d_%d"%(s,j))        
h = {}
for s in S:
    for j in Jiod:
        h[s,j] = m.addVar(vtype=GRB.CONTINUOUS,name="h_%d_%d"%(s,j))
p = {}
for j in Jiod:
    p[j] = m.addVar(vtype=GRB.CONTINUOUS,name="p_%d"%(j))
q = {}
for j in Jiod:
    q[j] = m.addVar(vtype=GRB.CONTINUOUS,name="q_%d"%(j))
mu = {}
for s in S:
    for j1 in Jiod:
        for j2 in Jiod:
            if j1 == j2:
                continue
            mu[s,j1,j2] = m.addVar(vtype=GRB.BINARY,name="mu_%d_%d_%d"%(s,j1,j2))
sigma = {}
for s in S:
    for j1 in Jiod:
        for j2 in Jiod:
            if j1 == j2:
                continue
            sigma[s,j1,j2] = m.addVar(vtype=GRB.BINARY,name="sigma_%d_%d_%d"%(s,j1,j2))
tau = {}
for s in S:
    for j1 in Jiod:
        for j2 in Jiod:
            if j1 == j2:
                continue
            tau[s,j1,j2] = m.addVar(vtype=GRB.BINARY,name="tau_%d_%d_%d"%(s,j1,j2))

In [11]:
m.setObjective(Cmax,GRB.MINIMIZE)

In [12]:
# for i in J+JI:
#     for j in J+JI:
#         for k in R:
#             if i==j:
#                 continue
#             else:
#                 m.addConstr(u[i,k] - u[j,k] + nn*z[k,i,j] <= nn-1)

# for i in J+JI:
#     for k in R:
#         m.addConstr(u[i,k] >= 0)
#         m.addConstr(u[i,k] <= nn-1)

In [13]:
# define Jod = J+Jo+Jd
# define Jo = J+Jo
# define Jd = J+Jd
# jo,jp
#3.2.1
for j in Jiod:
    m.addConstr(quicksum(x[s,j] for s in S) == 1)
    
for j in J:
    m.addConstr(x[S[0],j] == 0)
for j in iod:
    m.addConstr(x[rou[j],j] == 1)
#3.2.2
for j in J+JI:
    m.addConstr(quicksum(w[r,j] for r in R) == 1)


for j1 in Jio:
    for r in R:
        m.addConstr(quicksum(z[r,j1,j2] for j2 in Jid if j1 != j2) == w[r,j1])

# new
for j in JF:
    m.addConstr(quicksum(z[r,b[j],j] for r in R) == 1)

for j2 in Jid:
    for r in R:
        m.addConstr(quicksum(z[r,j1,j2] for j1 in Jio if j1 != j2) == w[r,j2])
        
for j1 in Jio:
    for j2 in Jid:
        for r in R:
            if j1 ==j2:
                m.addConstr(z[r,j,j]  == 0)

for s1 in S:
    j1 = jo[0]
    m.addConstr(quicksum(quicksum(v[s1,s2,j1,j2] for s2 in S) for j2 in Jid) == nr * x[s1,j1])

for s1 in S:
    for j1 in J+JI:
        m.addConstr(quicksum(quicksum(v[s1,s2,j1,j2] for s2 in S) for j2 in Jid) == x[s1,j1])    
    
for s2 in S:
    j2 = jd[0]
    m.addConstr(quicksum(quicksum(v[s1,s2,j1,j2] for s1 in S) for j1 in Jio) == nr * x[s2,b[j2]])

for s2 in S:
    for j2 in J:
        m.addConstr(quicksum(quicksum(v[s1,s2,j1,j2] for s1 in S) for j1 in Jio) == x[s2,b[j2]]) 
        
for j2 in JI:
    s2 = int(rou[j2])
    m.addConstr(quicksum(quicksum(v[s1,s2,j1,j2] for s1 in S) for j1 in Jio) == x[s2,j2]) 
    
for j1 in Jio:
    for j2 in Jid:
        m.addConstr(quicksum(quicksum(v[s1,s2,j1,j2] for s2 in S) for s1 in S) == quicksum(z[r,j1,j2] for r in R)) 

In [14]:
for j in Jiod:
    m.addConstr(p[j] >= 0)
    m.addConstr(p[j] <= M)
    m.addConstr(q[j] >= 0)
    m.addConstr(q[j] <= M)
    
for j in Jiod:
    for s in S:
        m.addConstr(g[s,j] >= 0)
        m.addConstr(g[s,j] <= M)
        m.addConstr(h[s,j] >= 0)
        m.addConstr(h[s,j] <= M)

In [15]:
# #define iod = ji+jo+jd
# #define od = jo+jd
# #define Jiod = j+ji+jo+jd

# #3.2.3
# # for k in K:
# #     for s in S:
# #         for j in listj[k][1:]:
# #             maxnum = np.maximum(q[j-1], x[s,b[j]]*(d[s,pi[j]] + h[s,b[j]]))
# #             m.addConstr(p[j] >= maxnum+Dsetup)

# for j in J:
#     m.addConstr(q[j] >= p[j] + D[j])

# # for j in J:
# #     for s in S:
# #         m.addConstr(q[s,j] >= x[s,j] * (d[pi[j],s]+q[j]) )

# # for s1 in S:
# #     for s2 in S:
# #         for j1 in Jp1:
# #             for j2 in Jp2:
# #                 m.addConstr(h[s2,b[j2]] >= v[s1,s2,j1,j2] * (d[s1,s2]+g[s1,j1]))
                
# for k in K:
#     for j in listJ[k][1:]:
#         m.addConstr(p[j] >= q[j-1]+Dsetup)

# for j in J:
#     for s in S:
#         m.addConstr(p[j] >= x[s,b[j]] * dsj[s,j] + h[s,b[j]] +Dsetup)        
        
# for j in J:
#     for s in S:
#         m.addConstr(g[s,j] >= djs[j,s] + q[j] + M*(x[s,j]-1))
        
# for s1 in S:
#     for s2 in S:
#         for j1 in Jo:
#             for j2 in Jd:
#                 m.addConstr(h[s2,b[j2]] >= dss[s1,s2]+g[s1,j1] +  M*(v[s1,s2,j1,j2] - 1))

# for s1 in S:
#     for s2 in S:
#         for j1 in Jio:
#             for j2 in Jd:
#                 m.addConstr(h[s2,b[j2]] >= (dss[s1,s2]+g[s1,j1])*v[s1,s2,j1,j2])
                
# for j in iod:
#     for s in S:
#         m.addConstr(g[s,j] == 0)

# for j in JI:
#     for s in S:
#         m.addConstr(g[s,j] <= M * x[s,j])
        
# for j in od:
#     m.addConstr(h[rou[j],j] == M)
    
# for j in JE:
#     for s in S:
#         m.addConstr(h[s,j] >= M * x[s,j] - 1)
        
# # 到20个约束

# for j1 in Jiod:
#     for j2 in Jiod:
#         for s in S:
#             if j1 == j2:
#                 continue
#             m.addConstr(1+ mu[s,j1,j2] >= x[s,j1] + x[s,j2])
            
# # for j1 in Jiod:
# #     for j2 in Jiod:
# #         for s in S:
# #             if j1 == j2:
# #                 continue
# #             m.addConstr(M * (sigma[s,j1,j2] - 1) <= g[s,j2] - h[s,j1] -mu[s,j1,j2])

# # for j1 in Jiod:
# #     for j2 in Jiod:
# #         for s in S:
# #             if j1 == j2:
# #                 continue
# #             m.addConstr(M * (tau[s,j1,j2] - 1) <= g[s,j1] - h[s,j2] -mu[s,j1,j2])
            
# # for j1 in Jiod:
# #     for j2 in Jiod:
# #         for s in S:
# #             if j1 == j2:
# #                 continue
# #             m.addConstr(sigma[s,j1,j2] + tau[s,j1,j2] == 1)
            
# #Objective
# for k in K:
#     temp = listJ[k][-1]
#     m.addConstr(Cmax >= q[temp])

In [16]:
#define iod = ji+jo+jd
#define od = jo+jd
#define Jiod = j+ji+jo+jd


for j in Jiod:
    m.addConstr(q[j] >= p[j] + D[j])
    
for j in Jiod:
    for s in S:
        m.addConstr(h[s,j] >= g[s,j])
    
for s in S:
    for j in JI:
        m.addConstr(g[s,j] >= (q[j])*x[s,j])
        
for s in S:
    for j in J:
        m.addConstr(g[s,j] >= (q[j]+dsj[s,j])*x[s,j])
    
for j in Jiod:
    for s in S:
        m.addConstr((x[s,j] == 0 ) >> (h[s,j] == 0 ))  
        m.addConstr((x[s,j] == 0 ) >> (g[s,j] == 0 ))
    
for j in JE:
    for s in S:
        m.addConstr((x[s,j] == 1 ) >> (h[s,j] == M ))
        
for j in od:
    for s in S:
        m.addConstr((x[s,j] == 1 ) >> (h[s,j] == M ))  

        
for s1 in S:
    for s2 in S:
        for j1 in Jio:
            for j2 in JI:
                m.addConstr(p[j2] >= (dss[s1,s2]+g[s1,j1])*v[s1,s2,j1,j2])
                
for s1 in S:
    for s2 in S:
        for j1 in Jio:
            for j2 in J:
                m.addConstr(p[j2] >= (dss[s1,s2]+dsj[s2,j2]+g[s1,j1])*v[s1,s2,j1,j2])
                
for k in K:
    for j in listJ[k][1:]:
        m.addConstr(p[j] >= q[j-1]+Dsetup)
        

for j in J:
    for s in S:
        m.addConstr(p[j] >= x[s,b[j]] * dsj[s,j] + h[s,b[j]] +Dsetup)    
        
for s1 in S:
    for s2 in S:
        for j1 in Jo:
            for j2 in Jd:
                m.addConstr(h[s2,b[j2]] >= (dss[s1,s2]+g[s1,j1]) * v[s1,s2,j1,j2])

for j1 in J+JI:
    for j2 in J+JI:
        for s in S[1:]:
            if j1 == j2:
                continue
            m.addConstr(1+ mu[s,j1,j2] >= x[s,j1] + x[s,j2])
            
# for j1 in J+JI:
#     for j2 in J+JI:
#         for s in S[1:]:
#             if j1 == j2:
#                 continue
#             m.addConstr((g[s,j1]-h[s,j2]-mu[s,j1,j2]) * (g[s,j2]-h[s,j1]-mu[s,j1,j2]) <= 0)
# m.Params.NonConvex = 2

MM = 1e8
for j1 in J+JI:
    for j2 in J+JI:
        for s in S[1:]:
            if j1 == j2:
                continue
            m.addConstr(MM * (sigma[s,j1,j2] - 1) <= g[s,j2] - h[s,j1] -mu[s,j1,j2])

for j1 in J+JI:
    for j2 in J+JI:
        for s in S[1:]:
            if j1 == j2:
                continue
            m.addConstr(MM * (tau[s,j1,j2] - 1) <= g[s,j1] - h[s,j2] -mu[s,j1,j2])
            
for j1 in J+JI:
    for j2 in J+JI:
        for s in S[1:]:
            if j1 == j2:
                continue
            m.addConstr(sigma[s,j1,j2] + tau[s,j1,j2] == 1)

#Objective
for k in K:
    temp = listJ[k][-1]
    m.addConstr(Cmax >= q[temp])

In [17]:
Jio

[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 0]

In [18]:
%%time
m.optimize()
m.write("TRE.lp")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 19063 rows, 181056 columns and 447143 nonzeros
Model fingerprint: 0x8882d314
Model has 174313 quadratic constraints
Model has 1054 general constraints
Variable types: 897 continuous, 180159 integer (180135 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
  GenCon rhs range [1e+03, 1e+03]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 17440 rows and 148746 columns
Presolve time: 1.65s
Presolved: 63745 rows, 63371 columns, 254222 nonzeros
Variable types: 30953 continuous, 32418 integer (32169 binary)
Deterministic concurrent LP optimizer: primal and dual simplex


In [19]:
w

{(0, 1): <gurobi.Var w_0_1 (value 0.0)>,
 (0, 2): <gurobi.Var w_0_2 (value 1.0)>,
 (0, 3): <gurobi.Var w_0_3 (value 1.0)>,
 (0, 4): <gurobi.Var w_0_4 (value 0.0)>,
 (0, 6): <gurobi.Var w_0_6 (value 0.0)>,
 (0, 7): <gurobi.Var w_0_7 (value 1.0)>,
 (0, 8): <gurobi.Var w_0_8 (value 1.0)>,
 (0, 9): <gurobi.Var w_0_9 (value 0.0)>,
 (0, 10): <gurobi.Var w_0_10 (value 1.0)>,
 (0, 11): <gurobi.Var w_0_11 (value 1.0)>,
 (0, 12): <gurobi.Var w_0_12 (value 0.0)>,
 (0, 13): <gurobi.Var w_0_13 (value 0.0)>,
 (0, 0): <gurobi.Var w_0_0 (value 1.0)>,
 (0, 5): <gurobi.Var w_0_5 (value 1.0)>,
 (1, 1): <gurobi.Var w_1_1 (value 1.0)>,
 (1, 2): <gurobi.Var w_1_2 (value 0.0)>,
 (1, 3): <gurobi.Var w_1_3 (value 0.0)>,
 (1, 4): <gurobi.Var w_1_4 (value 1.0)>,
 (1, 6): <gurobi.Var w_1_6 (value 1.0)>,
 (1, 7): <gurobi.Var w_1_7 (value 0.0)>,
 (1, 8): <gurobi.Var w_1_8 (value 0.0)>,
 (1, 9): <gurobi.Var w_1_9 (value 1.0)>,
 (1, 10): <gurobi.Var w_1_10 (value 0.0)>,
 (1, 11): <gurobi.Var w_1_11 (value 0.0)>,
 (1,

In [20]:
b

[-1, 6, 7, 8, 9, 0]

In [21]:
for s1 in S:
    for s2 in S:
        for j1 in Jio:
            for j2 in J:
                if(p[j2].x < (dss[s1,s2]+dsj[s2,j2]+g[s1,j1].x)*v[s1,s2,j1,j2].x):
                    print(111)

In [22]:
for s1 in S:
    for s2 in S:
        for j1 in Jio:
            for j2 in JI:
                if(p[j2].x < (dss[s1,s2]+g[s1,j1].x)*v[s1,s2,j1,j2].x):
                    print(p[j2].x - (dss[s1,s2]+g[s1,j1].x)*v[s1,s2,j1,j2].x)

-1.1901590823981678e-12
-0.0004948792136119096
-1.2789769243681803e-13


In [23]:
for s in S:
    for j in Jiod:
        if x[s,j].x > 0.5:
            print(f'任务 {j} 被还到 {s} ')

任务 6 被还到 0 
任务 7 被还到 1 
任务 2 被还到 2 
任务 8 被还到 2 
任务 9 被还到 3 
任务 10 被还到 4 
任务 11 被还到 5 
任务 12 被还到 6 
任务 1 被还到 7 
任务 13 被还到 7 
任务 4 被还到 8 
任务 3 被还到 11 
任务 0 被还到 30 
任务 5 被还到 30 


In [24]:
x[4,10].x

1.0

In [25]:
p[13].x >= (dss[6,7]+g[6,12].x)*v[6,7,12,13].x

True

In [26]:
dss[6,7]

2.0

In [27]:
temp = 0
print(x[5,2].x)
for s in S:
    for j in Jid:
        temp += v[5,s,2,j].x
print(temp)
#         if v[5,s,2,j].x > 0:
#             print(f'任务{j},位置{s}')

0.0
0.0


In [28]:
for s1 in S:
    for s2 in S:
        if v[s1,s2,3,13].x > 0:
            print(f'位置{s1},位置{s2}',{v[s1,s2,3,13].x})

In [29]:
x[10,2].x

-0.0

In [30]:
dss[6,7]

2.0

In [31]:
v[6,7,12,13].x

-0.0

In [32]:
z[0,12,13].x

-0.0

In [33]:
p[8].x

8.999443091258328

In [34]:
q[13].x

5.0

In [35]:
g[7,13].x

5.0

In [36]:
h[0,6].x

10.99999999999881

In [37]:
rou[13]

7.0

In [38]:
v[3,7,3,13].x

-0.0

In [39]:
for j in Jiod:
    for s in S:
        if h[s,j].x < 500 and x[s,j].x > 0.5:
            print(f'任务 {j} 离开位置 {s}在时刻{h[s,j].x} ')

任务 6 离开位置 0在时刻10.99999999999881 
任务 7 离开位置 1在时刻39.99944309125822 
任务 8 离开位置 2在时刻8.999443091258328 
任务 9 离开位置 3在时刻41.999443091258215 
任务 11 离开位置 5在时刻35.999443091258215 
任务 12 离开位置 6在时刻71.99944309125821 
任务 13 离开位置 7在时刻5.0 


In [40]:
for j in Jiod:
    for s in S:
        if g[s,j].x < 500:
            print(f'任务 {j} 占据位置 {s}在时刻{h[s,j].x} ')

任务 1 占据位置 0在时刻0.0 
任务 1 占据位置 1在时刻0.0 
任务 1 占据位置 2在时刻0.0005161000840416818 
任务 1 占据位置 3在时刻0.0 
任务 1 占据位置 4在时刻0.0 
任务 1 占据位置 5在时刻0.0 
任务 1 占据位置 6在时刻0.0 
任务 1 占据位置 7在时刻999.9994828277191 
任务 1 占据位置 8在时刻0.0 
任务 1 占据位置 9在时刻0.0 
任务 1 占据位置 10在时刻0.0 
任务 1 占据位置 11在时刻0.0 
任务 1 占据位置 12在时刻0.0 
任务 1 占据位置 13在时刻0.0 
任务 1 占据位置 14在时刻0.0 
任务 1 占据位置 15在时刻0.0 
任务 1 占据位置 16在时刻0.0 
任务 1 占据位置 17在时刻0.0 
任务 1 占据位置 18在时刻0.0 
任务 1 占据位置 19在时刻0.0 
任务 1 占据位置 20在时刻0.0 
任务 1 占据位置 21在时刻0.0 
任务 1 占据位置 22在时刻0.0 
任务 1 占据位置 23在时刻0.0 
任务 1 占据位置 24在时刻0.0 
任务 1 占据位置 25在时刻0.0 
任务 1 占据位置 26在时刻0.0 
任务 1 占据位置 27在时刻0.0 
任务 1 占据位置 28在时刻0.0 
任务 1 占据位置 29在时刻0.0 
任务 1 占据位置 30在时刻0.0 
任务 2 占据位置 0在时刻0.0 
任务 2 占据位置 1在时刻0.0 
任务 2 占据位置 3在时刻0.0 
任务 2 占据位置 4在时刻0.0 
任务 2 占据位置 5在时刻0.0 
任务 2 占据位置 6在时刻0.0 
任务 2 占据位置 7在时刻0.0 
任务 2 占据位置 8在时刻0.0 
任务 2 占据位置 9在时刻0.0 
任务 2 占据位置 10在时刻0.0 
任务 2 占据位置 11在时刻0.0 
任务 2 占据位置 12在时刻0.0 
任务 2 占据位置 13在时刻0.0 
任务 2 占据位置 14在时刻0.0 
任务 2 占据位置 15在时刻0.0 
任务 2 占据位置 16在时刻0.0 
任务 2 占据位置 17在时刻0.0 
任务 2 占据位置 18在时刻0.0 
任务 2 占据

In [41]:
for r in R:
    for j1 in Jio:
        for j2 in Jid:
            if z[r, j1, j2].x > 0.5:
                    print(f'机器人{r} 从任务 {j1} 到任务 {j2} 的路径被使用')

机器人0 从任务 2 到任务 10 的路径被使用
机器人0 从任务 3 到任务 11 的路径被使用
机器人0 从任务 7 到任务 2 的路径被使用
机器人0 从任务 8 到任务 3 的路径被使用
机器人0 从任务 10 到任务 5 的路径被使用
机器人0 从任务 11 到任务 7 的路径被使用
机器人0 从任务 0 到任务 8 的路径被使用
机器人1 从任务 1 到任务 9 的路径被使用
机器人1 从任务 4 到任务 12 的路径被使用
机器人1 从任务 6 到任务 1 的路径被使用
机器人1 从任务 9 到任务 4 的路径被使用
机器人1 从任务 12 到任务 5 的路径被使用
机器人1 从任务 13 到任务 6 的路径被使用
机器人1 从任务 0 到任务 13 的路径被使用


In [42]:
x

{(0, 1): <gurobi.Var x_0_1 (value 0.0)>,
 (0, 2): <gurobi.Var x_0_2 (value 0.0)>,
 (0, 3): <gurobi.Var x_0_3 (value 0.0)>,
 (0, 4): <gurobi.Var x_0_4 (value 0.0)>,
 (0, 6): <gurobi.Var x_0_6 (value 1.0)>,
 (0, 7): <gurobi.Var x_0_7 (value 0.0)>,
 (0, 8): <gurobi.Var x_0_8 (value 0.0)>,
 (0, 9): <gurobi.Var x_0_9 (value 0.0)>,
 (0, 10): <gurobi.Var x_0_10 (value 0.0)>,
 (0, 11): <gurobi.Var x_0_11 (value 0.0)>,
 (0, 12): <gurobi.Var x_0_12 (value 0.0)>,
 (0, 13): <gurobi.Var x_0_13 (value 0.0)>,
 (0, 0): <gurobi.Var x_0_0 (value 0.0)>,
 (0, 5): <gurobi.Var x_0_5 (value 0.0)>,
 (1, 1): <gurobi.Var x_1_1 (value 0.0)>,
 (1, 2): <gurobi.Var x_1_2 (value -0.0)>,
 (1, 3): <gurobi.Var x_1_3 (value -0.0)>,
 (1, 4): <gurobi.Var x_1_4 (value -0.0)>,
 (1, 6): <gurobi.Var x_1_6 (value 0.0)>,
 (1, 7): <gurobi.Var x_1_7 (value 1.0)>,
 (1, 8): <gurobi.Var x_1_8 (value 0.0)>,
 (1, 9): <gurobi.Var x_1_9 (value 0.0)>,
 (1, 10): <gurobi.Var x_1_10 (value 0.0)>,
 (1, 11): <gurobi.Var x_1_11 (value 0.0)>,
 

In [43]:
p

{1: <gurobi.Var p_1 (value 22.99999999999881)>,
 2: <gurobi.Var p_2 (value 49.99944309125822)>,
 3: <gurobi.Var p_3 (value 18.99944309125833)>,
 4: <gurobi.Var p_4 (value 51.999443091258215)>,
 6: <gurobi.Var p_6 (value 10.99999999999881)>,
 7: <gurobi.Var p_7 (value 39.99944309125822)>,
 8: <gurobi.Var p_8 (value 8.999443091258328)>,
 9: <gurobi.Var p_9 (value 41.999443091258215)>,
 10: <gurobi.Var p_10 (value 1000.0)>,
 11: <gurobi.Var p_11 (value 35.999443091258215)>,
 12: <gurobi.Var p_12 (value 71.99944309125821)>,
 13: <gurobi.Var p_13 (value 5.0)>,
 0: <gurobi.Var p_0 (value 0.0)>,
 5: <gurobi.Var p_5 (value 0.0)>}

In [44]:
q

{1: <gurobi.Var q_1 (value 29.99999999999881)>,
 2: <gurobi.Var q_2 (value 59.99944309125822)>,
 3: <gurobi.Var q_3 (value 23.99944309125833)>,
 4: <gurobi.Var q_4 (value 59.999443091258215)>,
 6: <gurobi.Var q_6 (value 10.99999999999881)>,
 7: <gurobi.Var q_7 (value 39.99944309125822)>,
 8: <gurobi.Var q_8 (value 8.999443091258328)>,
 9: <gurobi.Var q_9 (value 41.999443091258215)>,
 10: <gurobi.Var q_10 (value 1000.0)>,
 11: <gurobi.Var q_11 (value 35.999443091258215)>,
 12: <gurobi.Var q_12 (value 71.99944309125821)>,
 13: <gurobi.Var q_13 (value 5.0)>,
 0: <gurobi.Var q_0 (value 1000.0)>,
 5: <gurobi.Var q_5 (value 1000.0)>}

In [45]:
v

{(0, 0, 1, 1): <gurobi.Var v_0_0_1_1 (value 0.0)>,
 (0, 0, 1, 2): <gurobi.Var v_0_0_1_2 (value 0.0)>,
 (0, 0, 1, 3): <gurobi.Var v_0_0_1_3 (value 0.0)>,
 (0, 0, 1, 4): <gurobi.Var v_0_0_1_4 (value 0.0)>,
 (0, 0, 1, 6): <gurobi.Var v_0_0_1_6 (value 0.0)>,
 (0, 0, 1, 7): <gurobi.Var v_0_0_1_7 (value 0.0)>,
 (0, 0, 1, 8): <gurobi.Var v_0_0_1_8 (value 0.0)>,
 (0, 0, 1, 9): <gurobi.Var v_0_0_1_9 (value 0.0)>,
 (0, 0, 1, 10): <gurobi.Var v_0_0_1_10 (value 0.0)>,
 (0, 0, 1, 11): <gurobi.Var v_0_0_1_11 (value 0.0)>,
 (0, 0, 1, 12): <gurobi.Var v_0_0_1_12 (value 0.0)>,
 (0, 0, 1, 13): <gurobi.Var v_0_0_1_13 (value 0.0)>,
 (0, 0, 1, 5): <gurobi.Var v_0_0_1_5 (value 0.0)>,
 (0, 0, 2, 1): <gurobi.Var v_0_0_2_1 (value 0.0)>,
 (0, 0, 2, 2): <gurobi.Var v_0_0_2_2 (value 0.0)>,
 (0, 0, 2, 3): <gurobi.Var v_0_0_2_3 (value 0.0)>,
 (0, 0, 2, 4): <gurobi.Var v_0_0_2_4 (value 0.0)>,
 (0, 0, 2, 6): <gurobi.Var v_0_0_2_6 (value 0.0)>,
 (0, 0, 2, 7): <gurobi.Var v_0_0_2_7 (value 0.0)>,
 (0, 0, 2, 8): <gurobi.

In [46]:
dss

array([[ 0.,  2.,  4.,  6.,  8.,  2.,  4.,  6.,  8., 10.,  4.,  6.,  8.,
        10., 12.,  6.,  8., 10., 12., 14.,  8., 10., 12., 14., 16., 10.,
        12., 14., 16., 18., 11.],
       [ 2.,  0.,  2.,  4.,  6.,  4.,  2.,  4.,  6.,  8.,  6.,  4.,  6.,
         8., 10.,  8.,  6.,  8., 10., 12., 10.,  8., 10., 12., 14., 12.,
        10., 12., 14., 16.,  9.],
       [ 4.,  2.,  0.,  2.,  4.,  6.,  4.,  2.,  4.,  6.,  8.,  6.,  4.,
         6.,  8., 10.,  8.,  6.,  8., 10., 12., 10.,  8., 10., 12., 14.,
        12., 10., 12., 14.,  7.],
       [ 6.,  4.,  2.,  0.,  2.,  8.,  6.,  4.,  2.,  4., 10.,  8.,  6.,
         4.,  6., 12., 10.,  8.,  6.,  8., 14., 12., 10.,  8., 10., 16.,
        14., 12., 10., 12.,  9.],
       [ 8.,  6.,  4.,  2.,  0., 10.,  8.,  6.,  4.,  2., 12., 10.,  8.,
         6.,  4., 14., 12., 10.,  8.,  6., 16., 14., 12., 10.,  8., 18.,
        16., 14., 12., 10., 11.],
       [ 2.,  4.,  6.,  8., 10.,  0.,  2.,  4.,  6.,  8.,  2.,  4.,  6.,
         8., 10.,  4.,  6.,

In [47]:
h

{(0, 1): <gurobi.Var h_0_1 (value 0.0)>,
 (0, 2): <gurobi.Var h_0_2 (value 0.0)>,
 (0, 3): <gurobi.Var h_0_3 (value 0.0)>,
 (0, 4): <gurobi.Var h_0_4 (value 0.0)>,
 (0, 6): <gurobi.Var h_0_6 (value 10.99999999999881)>,
 (0, 7): <gurobi.Var h_0_7 (value 0.0)>,
 (0, 8): <gurobi.Var h_0_8 (value 0.0)>,
 (0, 9): <gurobi.Var h_0_9 (value 0.0)>,
 (0, 10): <gurobi.Var h_0_10 (value 0.0)>,
 (0, 11): <gurobi.Var h_0_11 (value 0.0)>,
 (0, 12): <gurobi.Var h_0_12 (value 0.0)>,
 (0, 13): <gurobi.Var h_0_13 (value 0.0)>,
 (0, 0): <gurobi.Var h_0_0 (value 0.0)>,
 (0, 5): <gurobi.Var h_0_5 (value 0.0)>,
 (1, 1): <gurobi.Var h_1_1 (value 0.0)>,
 (1, 2): <gurobi.Var h_1_2 (value 0.0)>,
 (1, 3): <gurobi.Var h_1_3 (value 0.0)>,
 (1, 4): <gurobi.Var h_1_4 (value 0.0)>,
 (1, 6): <gurobi.Var h_1_6 (value 0.0)>,
 (1, 7): <gurobi.Var h_1_7 (value 39.99944309125822)>,
 (1, 8): <gurobi.Var h_1_8 (value 0.0)>,
 (1, 9): <gurobi.Var h_1_9 (value 0.0)>,
 (1, 10): <gurobi.Var h_1_10 (value 0.0)>,
 (1, 11): <gurobi.V

In [48]:
g

{(0, 1): <gurobi.Var g_0_1 (value 0.0)>,
 (0, 2): <gurobi.Var g_0_2 (value 0.0)>,
 (0, 3): <gurobi.Var g_0_3 (value 0.0)>,
 (0, 4): <gurobi.Var g_0_4 (value 0.0)>,
 (0, 6): <gurobi.Var g_0_6 (value 10.99999999999881)>,
 (0, 7): <gurobi.Var g_0_7 (value 0.0)>,
 (0, 8): <gurobi.Var g_0_8 (value 0.0)>,
 (0, 9): <gurobi.Var g_0_9 (value 0.0)>,
 (0, 10): <gurobi.Var g_0_10 (value 0.0)>,
 (0, 11): <gurobi.Var g_0_11 (value 0.0)>,
 (0, 12): <gurobi.Var g_0_12 (value 0.0)>,
 (0, 13): <gurobi.Var g_0_13 (value 0.0)>,
 (0, 0): <gurobi.Var g_0_0 (value 0.0)>,
 (0, 5): <gurobi.Var g_0_5 (value 0.0)>,
 (1, 1): <gurobi.Var g_1_1 (value 0.0)>,
 (1, 2): <gurobi.Var g_1_2 (value 0.0)>,
 (1, 3): <gurobi.Var g_1_3 (value 0.0)>,
 (1, 4): <gurobi.Var g_1_4 (value 0.0)>,
 (1, 6): <gurobi.Var g_1_6 (value 0.0)>,
 (1, 7): <gurobi.Var g_1_7 (value 39.99944309125822)>,
 (1, 8): <gurobi.Var g_1_8 (value 0.0)>,
 (1, 9): <gurobi.Var g_1_9 (value 0.0)>,
 (1, 10): <gurobi.Var g_1_10 (value 0.0)>,
 (1, 11): <gurobi.V